In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import math
import matplotlib.pyplot as plt

In [178]:
# Read data
df = pd.read_csv('.\eth_2018_price_10T.csv',index_col=0)
df.head()

,mean_price,min_price,max_price
2017-12-31 19:00:00,739.337923,732.264535,746.00
2017-12-31 19:10:00,737.486161,729.949027,745.32
2017-12-31 19:20:00,736.453282,727.042308,744.16
2017-12-31 19:30:00,735.683226,728.656970,742.94
2017-12-31 19:40:00,734.582817,728.689437,739.08


In [179]:
# There are some rows that will cause error
error_value = np.unique(np.where(np.isfinite(df)==0)[0]).tolist()
error_value

[9978, 9979, 9980, 9981, 9982, 9983]

In [180]:
# Drop the error rows
df.drop(df.iloc[error_value].index, axis = 0, inplace = True)

# initialize params

In [181]:
N = 18 # batch size used to build model
M = 600 # timespan we use to train the model
score_threshold = 0.7 # rsrs threshold
# Parameters used to calculate moving average
mean_day = 20 # timespan to calculate the most recent close price
mean_diff_day = 3 # time difference to calculate the before close price


In [10]:
df.iloc[:M+N]

,mean_price,min_price,max_price
2017-12-31 19:00:00,739.337923,732.264535,746.000000
2017-12-31 19:10:00,737.486161,729.949027,745.320000
2017-12-31 19:20:00,736.453282,727.042308,744.160000
2017-12-31 19:30:00,735.683226,728.656970,742.940000
2017-12-31 19:40:00,734.582817,728.689437,739.080000
...,...,...,...
2018-01-05 01:10:00,995.418574,980.490000,1004.043402
2018-01-05 01:20:00,999.516768,986.606473,1013.863225
2018-01-05 01:30:00,1002.937961,991.929847,1016.107908
2018-01-05 01:40:00,1003.522279,991.720005,1020.239278


In [182]:
# main algorithm
def get_ols(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    r2 = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y, ddof=1)))
    return (intercept, slope, r2)

In [183]:
# Base slope
def initial_slope_series(start_time):
    data = df.iloc[start_time-(N + M):start_time]
    return [get_ols(data.min_price[i:i+N], data.max_price[i:i+N])[1] for i in range(M)]

In [184]:
# Calculate std rsrs score
def get_zscore(slope_series):
    mean = np.mean(slope_series)
    std = np.std(slope_series)
    return (slope_series[-1] - mean) / std

In [71]:
# pick a day to start
start_time = np.where(df.index=='2018-01-05 02:00:00')[0][0]

In [185]:
# leave the last one out to avoid duplication
# this is our base slop series
slope_series = initial_slope_series(start_time)[:-1]
len(slope_series)

599

In [186]:
get_zscore(slope_series)

0.8133157629700516

In [187]:
def get_timing_signal(start_time):
    #start_time = np.where(df.index==start_time)[0][0]
    # calculate MA signal
    close_data = df.iloc[start_time-(mean_day + mean_diff_day):start_time]
    # 23 days，take the last 20 days
    today_MA = close_data.mean_price[mean_diff_day:].mean() 
    # 23 days，take the first 20 days
    before_MA = close_data.mean_price[:-mean_diff_day].mean()
    # calculate rsrs signal
    high_low_data = pd.DataFrame({'low':df.iloc[start_time-N:start_time].min_price.to_list(),
                                  'high':df.iloc[start_time-N:start_time].max_price.to_list()})
    intercept, slope, r2 = get_ols(high_low_data.low, high_low_data.high)
    slope_series.append(slope)

    rsrs_score = get_zscore(slope_series[-M:]) * r2
    # use signal combo to get final signal
    if rsrs_score > score_threshold and today_MA > before_MA:
        return "BUY"
    elif rsrs_score < -score_threshold and today_MA < before_MA:
        return "SELL"
    else:
        return "NEUTRAL"

In [190]:
# test
get_timing_signal(-1)#'2018-01-05 02:00:00')

'NEUTRAL'

# Run the model

In [189]:
signal = df[-1].index.map(lambda index:get_timing_signal(index))
print(list(signal))

KeyError: -1

In [136]:
list(signal)

['KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 'KEEP',
 

In [138]:
#pd.DataFrame(['okfq-xbt-usd,14682.26,2,1514765115'],sep=',')

TypeError: __init__() got an unexpected keyword argument 'sep'

In [141]:
#pd.DataFrame('okfq-xbt-usd,14682.26,2,1514765115'.split(',')).T

,0,1,2,3
0,okfq-xbt-usd,14682.26,2,1514765115


In [153]:
datetime.fromtimestamp(np.array('okfq-xbt-usd,14682.26,2,1514765115'.split(',')).T[3].astype(int))

datetime.datetime(2017, 12, 31, 19, 5, 15)

In [156]:
#pd.DataFrame([[1],[2],[3],[4]]).append(pd.DataFrame(np.array('okfq-xbt-usd,14682.26,2,1514765115'.split(',')).T))

,0
0,1
1,2
2,3
3,4
0,okfq-xbt-usd
1,14682.26
2,2
3,1514765115


In [170]:
#pd.DataFrame('okfq-xbt-usd,14682.26,2,1514765115'.split(',')).T[[3]]

,3
0,1514765115


In [172]:
#datetime.fromtimestamp(1514764801)

datetime.datetime(2017, 12, 31, 19, 0, 1)

In [192]:
csv_rows = ['okfq-xbt-usd,14682.26,2,1514765115','okf1-xbt-usd,13793.65,2,1514765115','stmp-xbt-usd,13789.01,0.00152381,1514765115']

In [175]:
# csv_row =  pd.DataFrame('okfq-xbt-usd,14682.26,2,1514765115'.split(',')).T
# csv_row[[3]] = csv_row[[3]].apply(datetime.fromtimestamp)
# csv_row.set_index('3')
# # df = df.append(csv_row, ignore_index=True)
# csv_row

KeyError: "None of ['3'] are in the columns"

In [177]:
#pd.DataFrame(columns = ['pair','price','amount','time_stamp'])

,pair,price,amount,time_stamp


In [222]:
tmp = pd.DataFrame(columns = ['pair','price','amount','time_stamp'])
tmp

,pair,price,amount,time_stamp


In [223]:
for csv_row in csv_rows:
    csv_row =  pd.DataFrame(csv_row.split(',')).T
    csv_row.rename(columns={0:'pair',1:'price',2:'amount',3:'time_stamp'},inplace = True)
    tmp = pd.concat([tmp,csv_row], ignore_index=True,axis = 0)

In [224]:
# csv_row = pd.DataFrame(csv_rows[0].split(',')).T
# csv_row[3] = csv_row[[3]].apply(datetime.fromtimestamp)
csv_row

,pair,price,amount,time_stamp
0,stmp-xbt-usd,13789.01,0.00152381,1514765115


In [225]:
tmp

,pair,price,amount,time_stamp
0,okfq-xbt-usd,14682.26,2,1514765115
1,okf1-xbt-usd,13793.65,2,1514765115
2,stmp-xbt-usd,13789.01,0.00152381,1514765115


In [226]:
counter = 0
for i in range(100):
    if counter < 20:
        counter +=1
        print(counter)
    else:
        counter = 0
        print(counter)
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [227]:
def set_parameter():
    g.N = 18